Merging The Two Bronze Layers

In [0]:
%sql
-- Step 1: Create temp view from merged Box files with governance overrides
CREATE OR REPLACE TEMP VIEW contracts_base_temp AS
SELECT
  COALESCE(high.Contract_ID, one_m.Contract_ID) AS Contract_ID,
  COALESCE(high.BuyingHub_ID, one_m.BuyingHub_ID) AS BuyingHub_ID,
  COALESCE(high.Total_Contract_Cost, one_m.Total_Contract_Cost) AS Total_Contract_Cost,
  COALESCE(high.Term_Start_Date, one_m.Term_Start_Date) AS Term_Start_Date,
  COALESCE(high.Term_End_Date, one_m.Term_End_Date) AS Term_End_Date,
  COALESCE(high.Contract_Lead, one_m.Contract_Lead) AS Contract_Lead,
  COALESCE(high.Future_Status, one_m.Future_Status) AS Future_Status,
  COALESCE(high.Domain_Acronym, one_m.Domain_Acronym) AS Domain_Acronym,
  COALESCE(high.TechMod_ID, one_m.TechMod_ID) AS TechMod_ID,
  COALESCE(high.TechMod_Workflow_Status, one_m.TechMod_Workflow_Status) AS TechMod_Workflow_Status,
  COALESCE(high.Sweet_Sixteen, one_m.Sweet_Sixteen) AS Sweet_Sixteen,
  COALESCE(high.Strategy_List, one_m.Strategy_List) AS Strategy_List,
  COALESCE(high.Current_Status, one_m.Current_Status) AS Current_Status,
  COALESCE(high.Vendor, one_m.Vendor) AS Vendor,
  COALESCE(high.Description, one_m.Description) AS Description,
  COALESCE(high.Governance_Required, one_m.Governance_Required) AS Governance_Required,
  COALESCE(high.Governance_Date, one_m.Governance_Date) AS Governance_Date,
  COALESCE(high.Comments, one_m.Comments) AS Comments,

  -- Days from end date logic
  CASE
    WHEN CAST(COALESCE(high.Term_End_Date, one_m.Term_End_Date) AS DATE) < CURRENT_DATE THEN 'Already ended'
    WHEN DATEDIFF(CAST(COALESCE(high.Term_End_Date, one_m.Term_End_Date) AS DATE), CURRENT_DATE) BETWEEN 0 AND 30 THEN 'Next 30 days'
    WHEN DATEDIFF(CAST(COALESCE(high.Term_End_Date, one_m.Term_End_Date) AS DATE), CURRENT_DATE) BETWEEN 31 AND 60 THEN 'Next 60 days'
    WHEN DATEDIFF(CAST(COALESCE(high.Term_End_Date, one_m.Term_End_Date) AS DATE), CURRENT_DATE) BETWEEN 61 AND 90 THEN 'Next 90 days'
    WHEN DATEDIFF(CAST(COALESCE(high.Term_End_Date, one_m.Term_End_Date) AS DATE), CURRENT_DATE) BETWEEN 91 AND 180 THEN 'Next 180 days'
    ELSE 'Beyond 180 days'
  END AS Days_From_End_Date,

  -- Governance override logic
  CASE
    WHEN COALESCE(high.Governance, one_m.Governance) IS NOT NULL
      AND COALESCE(high.Governance, one_m.Governance) != 'nan'
      THEN COALESCE(high.Governance, one_m.Governance)
    WHEN LOWER(COALESCE(high.Governance_Required, one_m.Governance_Required)) = 'no' THEN 'None'
    WHEN CAST(COALESCE(high.Total_Contract_Cost, one_m.Total_Contract_Cost) AS DOUBLE) > 5000000 THEN 'CPMO'
    WHEN CAST(COALESCE(high.Total_Contract_Cost, one_m.Total_Contract_Cost) AS DOUBLE) BETWEEN 1000000 AND 5000000 THEN 'TPMO'
    ELSE 'None'
  END AS Governance,

  -- Governance label override logic
  CASE
    WHEN COALESCE(high.Governance, one_m.Governance) = 'CPMO' THEN '+5 Million'
    WHEN COALESCE(high.Governance, one_m.Governance) = 'TPMO' THEN '1 to 5 Million'
    WHEN COALESCE(high.Governance, one_m.Governance) = 'None' THEN 'Below 1 Million'
    WHEN CAST(COALESCE(high.Total_Contract_Cost, one_m.Total_Contract_Cost) AS DOUBLE) > 5000000 THEN '+5 Million'
    WHEN CAST(COALESCE(high.Total_Contract_Cost, one_m.Total_Contract_Cost) AS DOUBLE) BETWEEN 1000000 AND 5000000 THEN '1 to 5 Million'
    ELSE 'Below 1 Million'
  END AS Governance_Label

FROM your_schema.game_high_cost_active AS high
FULL OUTER JOIN your_schema.game_1m_cost_active AS one_m
  ON high.Contract_ID = one_m.Contract_ID
WHERE COALESCE(high.Term_End_Date, one_m.Term_End_Date) IS NOT NULL;

-- Step 2: Display the temp view
SELECT * FROM contracts_base_temp;


Final Silver Table adding the suplemental data from airtable

Print

In [0]:
%sql
-- Create or replace table with interim contracts joined to Airtable data
CREATE OR REPLACE TABLE your_schema.interim_contracts_silver AS
SELECT
  base.*,
  air.Domain_Leader,
  air.Org_Leader
FROM (
  SELECT * FROM contracts_base_temp
  WHERE Contract_ID IS NOT NULL AND Contract_ID != 'TBD'
) AS base
LEFT JOIN your_schema.airtable_contracts_bronze AS air
  ON base.Contract_ID = air.Contract_ID;
